# Fake news detector

I want to check if X 

## Environment

In [205]:
# Basic
import numpy as np
import pandas as pd
import re

# NLP
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

#Vectorizer storage
import pickle

# Misc
import time
import progressbar

In [129]:
news_df1 = pd.read_csv('data/Fake.csv')
news_df2 = pd.read_csv('data/True.csv')
news_df3 = pd.read_csv('data/train.csv')
#news_df4 = pd.read_csv('data/test.csv')

## Data Wrangling

In [130]:
news_df1.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [131]:
news_df2.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [132]:
news_df3.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [133]:
news_df1.loc[:,'label'] = 'Fake'
news_df2.loc[:,'label'] = 'Real'

In [134]:
news_df1.drop(columns = ["subject", 'date'], inplace = True)

In [135]:
news_df2.drop(["subject", 'date'], axis = 1, inplace = True)

In [136]:
news_df3.drop(["author", 'id'], axis = 1, inplace = True)
news_df3

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1


In [137]:
news_df3['label'] = news_df3['label'].map({0: 'Real', 1: 'Fake'})
news_df3

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,Real
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",Fake
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,Fake
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,Fake
...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,Real
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,Real
20797,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,Real
20798,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",Fake


In [138]:
news_df3.head()

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,Real
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",Fake
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,Fake
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,Fake


In [141]:
news = pd.concat([news_df1, news_df2, news_df3]).reset_index(drop = True)

In [142]:
news

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,Fake
...,...,...,...
65693,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,Real
65694,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,Real
65695,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,Real
65696,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",Fake


In [143]:
news.to_csv('news dataframe')

In [144]:
news['label'].value_counts()

Fake    33894
Real    31804
Name: label, dtype: int64

In [146]:
# 
news = news.replace(' s ', 's ', regex=True)
news = news.replace(' t ', 't ', regex=True)
news = news.replace('2017', '2017 ', regex=True)

In [147]:
news.dropna(inplace=True)

In [148]:
news

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldnt wish all Americans a...,Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,Fake
...,...,...,...
65693,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,Real
65694,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,Real
65695,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,Real
65696,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",Fake


In [28]:
news['text'][0]

'Donald Trump just couldnt wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldnt do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017 Trumps tweet went down about as welll as you d expect.What kind of president sends a New Years greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency wont even a

In [29]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    return keywords

In [ ]:
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(news['text'])
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
X_train.head()

In [ ]:
vectoriser

# Test

In [30]:
test_news = news[64000:64100]

In [33]:
test_news

,title,text,label
64563,Trump llama a mantenimiento porque el botón nu...,Trump llama a mantenimiento porque el botón nu...,Fake
64564,"Wells Fargo Offers Regrets, but Doesn’t Admit ...",Wells Fargo was flowing with regrets on Friday...,Real
64566,No Proof Russia Hacked Hillary's E-mail,"Email \n""The Russian government has engaged in...",Fake
64567,"Leftist Philosopher Predicts ""Immediate Riots""...",French philosopher Michel Onfray claims if ...,Real
64568,Taser International Dominates the Police Body ...,When Micah Johnson went on a deadly shooting r...,Real
...,...,...,...
64662,Заседание Международного дискуссионного клуба ...,Заседание Международного дискуссионного клуба ...,Fake
64663,Se reencuentran dos gemeliers separados al nacer,Se reencuentran dos gemeliers separados al nac...,Fake
64664,Desperate Obama RIPS Comey: 'We Don't Operate ...,Desperate Obama RIPS Comey: 'We Don't Operate ...,Fake
64666,FranceInfo Changes Text on Its Website About “...,"October 29, 2016 Before: The tears of a 16-yea...",Fake


In [34]:
# Create an instance of RegexpTokenizer for alphanumeric tokens
tokeniser = RegexpTokenizer(r'\w+')
# Tokenise 'part1' string
tokens = tokeniser.tokenize(news['text'][0])
print(tokens)

['Donald', 'Trump', 'just', 'couldnt', 'wish', 'all', 'Americans', 'a', 'Happy', 'New', 'Year', 'and', 'leave', 'it', 'at', 'that', 'Instead', 'he', 'had', 'to', 'give', 'a', 'shout', 'out', 'to', 'his', 'enemies', 'haters', 'and', 'the', 'very', 'dishonest', 'fake', 'news', 'media', 'The', 'former', 'reality', 'show', 'star', 'had', 'just', 'one', 'job', 'to', 'do', 'and', 'he', 'couldnt', 'do', 'it', 'As', 'our', 'Country', 'rapidly', 'grows', 'stronger', 'and', 'smarter', 'I', 'want', 'to', 'wish', 'all', 'of', 'my', 'friends', 'supporters', 'enemies', 'haters', 'and', 'even', 'the', 'very', 'dishonest', 'Fake', 'News', 'Media', 'a', 'Happy', 'and', 'Healthy', 'New', 'Year', 'President', 'Angry', 'Pants', 'tweeted', '2018', 'will', 'be', 'a', 'great', 'year', 'for', 'America', 'As', 'our', 'Country', 'rapidly', 'grows', 'stronger', 'and', 'smarter', 'I', 'want', 'to', 'wish', 'all', 'of', 'my', 'friends', 'supporters', 'enemies', 'haters', 'and', 'even', 'the', 'very', 'dishonest', 

In [149]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)

In [150]:
X_train = vectoriser.fit_transform(news['text'][:55000])

In [151]:
LR = LogisticRegression()
LR.fit(X_train, news["label"][:55000].map({"Fake" : 1, "Real" : 0}))

LogisticRegression()

In [197]:
pickle.dump(LR, open("fake_detector", "wb"))
pickle.dump(vectoriser, open("vectoriser", "wb"))

In [199]:
fake_detector = pickle.load(open("fake_detector", "rb"))
vectoriser_1 = pickle.load(open("vectoriser", "rb"))

In [204]:
LR.score(vectoriser_1.transform(news['text'][55000:]), news["label"][55000:].map({"Fake" : 1, "Real" : 0}))

0.8802098802098802

In [88]:
#to_predict = vectoriser.transform([test_news["text"].loc[64563]])
predicted = LR.predict(vectoriser.transform(test_news["text"]))

In [109]:
LR.score(vectoriser.transform(sample["text"][:10]), sample["label"].map({"Fake" : 1, "Real" : 0})[:10])

0.7

In [70]:
test_news["label"].map({"Fake" : 1, "Real" : 0})

64563    1
64564    0
64566    1
64567    0
64568    0
        ..
64662    1
64663    1
64664    1
64666    1
64667    0
Name: label, Length: 100, dtype: int64

In [106]:
sample = news.sample(100)

In [35]:
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(test_news['text'])
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
    
X_train.columns

KeyboardInterrupt: 

In [ ]:
X_train

In [ ]:
test_news['label']

In [ ]:
test_news['label'] = np.where(test_news['label']=='Fake', 1, 0)
test_news['label'].value_counts()

# Web scrape cnn & fox news for news in the last 24h

# Proportion hypothesis testing